## Video Load

In [62]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [63]:
"""
contours, hierarchy = cv2.findContours(ROI, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                             
contour = contours[0]

epsilon = 0.005 * cv2.arcLength(contour, True)

approx = cv2.approxPolyDP(contour, epsilon, True)

result = cv2.copyTo(frameS04, ROI)

cv2.drawContours(frameS04, [approx], -1, (0,255,0), 1)
"""

'\ncontours, hierarchy = cv2.findContours(ROI, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)\n                             \ncontour = contours[0]\n\nepsilon = 0.005 * cv2.arcLength(contour, True)\n\napprox = cv2.approxPolyDP(contour, epsilon, True)\n\nresult = cv2.copyTo(frameS04, ROI)\n\ncv2.drawContours(frameS04, [approx], -1, (0,255,0), 1)\n'

In [64]:
def get_polygon_mask(shape, vertices):
    mask = np.zeros(shape, dtype=np.uint8)
    # 다각형 영역 채우기
    cv2.fillPoly(mask, vertices, 255)

    return mask

In [68]:
def load_video(path, ms=25):
    
    # Video Capture 객체 생성
    capture = cv2.VideoCapture(path)

    run, frame = capture.read() # 다음 Frame 읽기
    
    prevS04, prevS16, prevS64 = None, None, None
    
    if run: # Frame을 읽은 경우
        frameS04 = cv2.resize(frame,    (0, 0), fx=1/4, fy=1/4, interpolation= cv2.INTER_AREA)
        frameS16 = cv2.resize(frameS04, (0, 0), fx=1/4, fy=1/4, interpolation= cv2.INTER_AREA)
        frameS64 = cv2.resize(frameS16, (0, 0), fx=1/4, fy=1/4, interpolation= cv2.INTER_AREA)

        prevS04, prevS16, prevS64 = frameS04, frameS16, frameS64

    while capture.isOpened(): # Video Capture가 준비되었는지 확인
        
        run, frame = capture.read() # 다음 Frame 읽기
        
        if run: # Frame을 읽은 경우
            # 영상 사이즈 기준 1920 x 1080

            # 영상의 사이즈 조정 (x1/16 또는 x1/32)
            frameS04 = cv2.resize(frame,    (0, 0), fx=1/4, fy=1/4, interpolation= cv2.INTER_AREA)
            frameS16 = cv2.resize(frameS04, (0, 0), fx=1/4, fy=1/4, interpolation= cv2.INTER_AREA)
            frameS64 = cv2.resize(frameS16, (0, 0), fx=1/4, fy=1/4, interpolation= cv2.INTER_AREA)
                            
            hsv = cv2.cvtColor(frameS04, cv2.COLOR_BGR2HSV)

            h, s, v = cv2.split(hsv); 
 
            cv2.imshow("h", h)
            cv2.imshow("s", s)
            cv2.imshow("v", v)

            frameKMeans = h

            # 축구장 대표 컬러 추출
            data = frameKMeans.reshape(-1).astype(np.float32)

            criteria = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS, 10, 0.001)

            max_labels = 4

            compactness, labels, centers = cv2.kmeans(data, max_labels, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
            
            height, width = frameKMeans.shape

            # 관심 영역만 남기기
            vertices = np.array([[(0, height), (width // 2, 0), (width, height)]], dtype=np.int32)

            mask = get_polygon_mask((height, width), vertices)

            # 색상에 대한 히스토그램 구하기
            hist = cv2.calcHist([(labels.reshape(height, width).astype(np.uint8))], [0], mask, [max_labels], [0, max_labels])

            color = centers[np.argmax(hist)] 
            # 군집화 결과를 이용하여 출력 영상 생성
            centers = centers.astype(np.uint8)
            
            # 중심점 좌표를 받아서 dst에 입력 (307200, 3) 3은 중심 좌표
            dst = centers[labels.flatten()] # 각 픽셀을 K개 군집 중심 색상으로 치환
            
            # 입력 영상과 동일한 형태로 변환 (640,480,3)
            dst = dst.reshape((frameKMeans.shape))

            cv2.imshow("dst", dst)

            """
            # 축구장 ROI 평면 추출
            r = 24

            ROI = cv2.inRange(frameKMeans, color - [r] * 3, color + [r] * 3)

            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
            field_mask = cv2.morphologyEx(ROI, cv2.MORPH_ERODE, kernel)
            
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (19, 19))
            ROI = cv2.morphologyEx(field_mask, cv2.MORPH_DILATE, kernel)
            
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
            ROI = cv2.morphologyEx(ROI, cv2.MORPH_ERODE, kernel)

            cv2.imshow("dst", )

            #result = cv2.copyTo(frameS16, np.bitwise_and(ROI, np.bitwise_not(Segmentation)))

            result = cv2.copyTo(frameKMeans, ROI)

            field = cv2.copyTo(frameKMeans, field_mask)
            blurred = cv2.GaussianBlur(result, (7, 7), sigmaX=0, sigmaY=0)

            canny = cv2.Canny(blurred, 16, 48)

            cv2.imshow("ROI", ROI)
            cv2.imshow("result", result)
            cv2.imshow("frame", frameS04)
            cv2.imshow("canny", canny)
            cv2.waitKey(ms) # Millisecond 단위로 대기
            """
            prevS04, prevS16, prevS64 = frameS04, frameS16, frameS64
        else: # 재생이 완료되어 더 이상 Frame을 읽을 수 없는 경우
            break

    capture.release() # Capture 자원 반납
    cv2.destroyAllWindows() # 창 제거

In [69]:
# Video가 저장된 경로 입력
PATH = r"soccer1.mp4"

# Video 재생 및 반환 (Numpy Array)
load_video(PATH, ms=20)

KeyboardInterrupt: 